In [1]:
from ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
train = IPSL_DCPP('train',generate_statistics=False,lead_time_months=1)
# train_dataloader = torch.utils.data.DataLoader(train,batch_size=2,shuffle=True,num_workers=1,)


In [2]:
from pangu import PanguWeather
model = PanguWeather(soil=True)

/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")


In [4]:
train_dataloader = torch.utils.data.DataLoader(IPSL_DCPP('train',cfg.experiment.lead_time_months),batch_size=1,shuffle=False,num_workers=1)
batch = next(iter(train_dataloader))

In [5]:
model = hydra.utils.instantiate(cfg.experiment.module,backbone=hydra.utils.instantiate(cfg.experiment.backbone),dataset=train_dataloader.dataset)


In [6]:
trainer = pl.Trainer(fast_dev_run=2)
output = trainer.fit(model, train_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 2 batch(es). Logging and checkpointing is suppressed.
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defi

configure optimizers


/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] before torch.Size([1, 192, 17, 36, 36])
torch.Size([1, 135, 143, 144]) torch.Size([1, 135, 143, 144])
time_embedding.mlp.0.weight
time_embedding.mlp.0.bias
time_embedding.mlp.2.weight
time_embedding.mlp.2.bias
patchembed2d.proj.weight
patchembed2d.proj.bias
plev_patchembed3d.proj.weight
plev_patchembed3d.proj.bias
depth_patchembed3d.proj.weight
depth_patchembed3d.proj.bias
layer1.blocks.0.norm1.weight
layer1.blocks.0.norm1.bias
layer1.blocks.0.attn.earth_position_bias_table
layer1.blocks.0.attn.qkv.weight
layer1.blocks.0.attn.qkv.bias
layer1.blocks.0.attn.proj.weight
layer1.blocks.0.attn.proj.bias
layer1.blocks.0.norm2.weight
layer1.blocks.0.norm2.bias
layer1.blocks.0.mlp.fc1.weight
layer1.blocks.0.mlp.fc1.bias
layer1.blocks.0.mlp.fc2.weight
layer1.blocks.0.mlp.fc2.bias
layer1.blocks.1.norm1.weight
layer1.blocks.1.norm1.bias
layer1.blocks.1.attn.earth_position_bias_table
layer1.blocks.1.attn.qkv.weight
layer1.blocks.1.attn.qkv.bias
layer1.

`Trainer.fit` stopped: `max_steps=2` reached.


Epoch 0: 100%|██████████| 2/2 [00:21<00:00,  0.09it/s, train_loss=10.00]


In [5]:
out = model.forward(batch)

torch.Size([2, 192, 17, 36, 36])
before torch.Size([2, 192, 17, 36, 36])
after conv torch.Size([2, 320, 144, 144])
torch.Size([2, 8, 19, 144, 144]) torch.Size([2, 135, 1, 144, 144]) torch.Size([2, 3, 11, 144, 144])


In [46]:
import torch.nn as nn
from patch_recovery import Interpolate
from torch.nn import GELU as GeLU

class PatchRecovery3(nn.Module):
    def __init__(self, 
        input_dim=None,
        dim=192,
        downfactor=4,
        output_dim=137):
# input dim equals input_dim*z since we will be flattening stuff ?
        super().__init__()
        self.downfactor = downfactor
        if input_dim is None:
            input_dim = 8*dim
        self.head1 = nn.Sequential(
            nn.Conv2d(input_dim, 8*dim, kernel_size=1, stride=1, padding=0),
            Interpolate(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(8*dim, 4*dim, kernel_size=3, stride=1, padding=1),
            GeLU(),
            nn.Conv2d(4*dim, 4*dim, kernel_size=1, stride=1, padding=0))
        self.head2 = nn.Sequential(
            nn.GroupNorm(num_groups=32, num_channels=4*dim, eps=1e-6, affine=True),
            Interpolate(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(4*dim, 4*dim, kernel_size=3, stride=1, padding=1),
            GeLU())
        self.proj = nn.Conv2d(4*dim, output_dim, kernel_size=(4,5), stride=1, padding=(1,2))

    def forward(self, x):
        print('before',x.shape)
        #x = x.permute(0, 2, 1)
        #x = x.reshape((x.shape[0], x.shape[1]*Z, H, W))
        x = x.flatten(1, 2)
        x = self.head1(x)
        if self.downfactor == 4:
            x = self.head2(x)
        x = self.proj(x) 
        print('after conv',x.shape)
        output_surface = x[:, :135]
        output = x[:, 135:287].reshape((x.shape[0], 8, 19, *x.shape[-2:]))
        output_depth = x[:,287:].reshape((x.shape[0],3,11,*x.shape[-2:]))
        
        return output, output_surface.unsqueeze(-3),output_depth


In [47]:
zdim = 17
out_dim = 192
patch_size = (2,4,4)
patchrecovery = PatchRecovery3(input_dim=zdim*out_dim, output_dim=(135 * 1) + (8 * 19) + (3 * 11), downfactor=patch_size[-1])


In [48]:
import torch
x = torch.rand([2, 192, 17, 36, 36])

In [49]:
patchrecovery(x)

before torch.Size([2, 192, 17, 36, 36])
after conv torch.Size([2, 320, 143, 144])


(tensor([[[[[-2.7064e-02, -1.4669e-02,  7.2190e-02,  ...,  8.2308e-02,
              8.1953e-02,  5.2094e-02],
            [-9.2489e-02, -2.6002e-02, -5.2347e-02,  ...,  6.8183e-03,
              2.3710e-02, -6.3496e-02],
            [-1.4170e-01, -3.5914e-02, -9.2630e-02,  ..., -2.8282e-02,
              1.7360e-02, -5.6913e-02],
            ...,
            [-8.2917e-02, -6.0841e-02, -1.6290e-01,  ..., -2.3437e-01,
             -9.9697e-02, -1.2223e-01],
            [-1.2173e-01, -6.6633e-02, -1.7556e-01,  ..., -2.9638e-01,
             -2.1067e-01, -2.1983e-01],
            [-1.3214e-01, -6.6909e-02, -1.4989e-01,  ..., -2.6890e-01,
             -1.7432e-01, -1.8743e-01]],
 
           [[ 1.1772e-01,  1.1073e-01,  9.9348e-02,  ..., -3.3367e-03,
             -2.5156e-02, -2.7133e-02],
            [ 1.7336e-01,  1.6080e-01,  1.6879e-01,  ...,  1.8223e-02,
             -5.1440e-02, -5.2671e-02],
            [ 2.1734e-01,  2.1841e-01,  1.8820e-01,  ..., -4.0264e-02,
             -9.1693e